In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [20]:
# Configuración inicial
train_dir = 'data/train'
valid_dir = 'data/valid'
test_dir = 'data/test'
img_size = (300, 300)  # EfficientNetB3 recomienda 300x300
batch_size = 32

In [21]:
# Crear generadores de datos con augmentación moderada
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=[0.9, 1.1],
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 17019 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


In [22]:
# Construir el modelo con EfficientNet-B3
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=img_size + (3,))
base_model.trainable = False  # Congelar el modelo base para el entrenamiento inicial

# Añadir capas superiores personalizadas
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)  # Ajustar el Dropout a un 50% para prevenir sobreajuste
x = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001))(x)

# Crear el modelo completo
model = models.Model(inputs=base_model.input, outputs=x)

In [23]:
# Definir un Exponential Decay Scheduler
initial_learning_rate = 1e-4
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Configurar el optimizador con el scheduler
optimizer = Adam(learning_rate=lr_schedule)

# Compilar el modelo
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Definir los pesos de las clases (balanceados)
class_weights = {0: 1.0, 1: 1.0}

# Función para obtener el último epoch completado
def get_last_epoch():
    epoch_files = [int(f.split('_')[-1].split('.')[0]) for f in os.listdir() if f.startswith("backup_epoch")]
    return max(epoch_files) if epoch_files else 0

# Configurar los callbacks
checkpoint = ModelCheckpoint("fine_tuned_model.weights.h5", monitor='val_loss', save_best_only=True, mode='min', save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Callback adicional para respaldos de pesos y retomar el entrenamiento desde el último epoch
class BackupCheckpoint(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        self.model.save_weights(f"backup_epoch_{epoch+1}.weights.h5")

backup_checkpoint = BackupCheckpoint()

In [24]:
# Obtener el último epoch completado y cargar pesos
initial_epoch = get_last_epoch()

if initial_epoch > 2:
    print(f"Resuming training from epoch {initial_epoch}")
    model.load_weights(f"backup_epoch_{initial_epoch}.weights.h5")

# Entrenamiento inicial (antes de Fine-Tuning)
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    initial_epoch=initial_epoch,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, backup_checkpoint]
)

Epoch 3/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


532/532 ━━━━━━━━━━━━━━━━━━━━ 4254s 8s/step - accuracy: 0.4917 - loss: 0.7034 - val_accuracy: 0.5000 - val_loss: 0.6952 - learning_rate: 1.0000e-04
Epoch 4/20
532/532 ━━━━━━━━━━━━━━━━━━━━ 3436s 6s/step - accuracy: 0.4919 - loss: 0.7027 - val_accuracy: 0.5000 - val_loss: 0.6956 - learning_rate: 1.0000e-04
Epoch 5/20


2024-09-01 15:57:21.924120: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:16: Filling up shuffle buffer (this may take a while): 6 of 8
2024-09-01 15:57:25.067695: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


532/532 ━━━━━━━━━━━━━━━━━━━━ 3529s 7s/step - accuracy: 0.5005 - loss: 0.6995 - val_accuracy: 0.5000 - val_loss: 0.6951 - learning_rate: 1.0000e-04
Epoch 6/20
532/532 ━━━━━━━━━━━━━━━━━━━━ 3077s 6s/step - accuracy: 0.5036 - loss: 0.6988 - val_accuracy: 0.5000 - val_loss: 0.6955 - learning_rate: 1.0000e-04
Epoch 7/20
532/532 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4968 - loss: 0.7005

KeyboardInterrupt: 

In [ ]:
# Descongelar algunas capas del modelo base para fine-tuning
base_model.trainable = True

# Re-compilar el modelo con una tasa de aprendizaje más baja
fine_tune_lr = 1e-5
model.compile(optimizer=Adam(learning_rate=fine_tune_lr), loss='binary_crossentropy', metrics=['accuracy'])

# Continuar entrenamiento con ajuste fino
fine_tune_epochs = 10
history_fine = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=initial_epoch + fine_tune_epochs,
    initial_epoch=initial_epoch,
    class_weight=class_weights,
    callbacks=[checkpoint, early_stopping, reduce_lr, backup_checkpoint]
)

In [ ]:
# Guardar el modelo después del ajuste fino
model.save('fine_tuned_model_final.keras')

In [ ]:
# Evaluar el modelo en el conjunto de prueba
evaluation = model.evaluate(test_generator)
print(f'Test Loss: {evaluation[0]:.4f}')
print(f'Test Accuracy: {evaluation[1]*100:.2f}%')

# Obtener predicciones para cada imagen en el conjunto de prueba
predictions = model.predict(test_generator, verbose=1)

# Convertir las predicciones a clases
predicted_classes = (predictions > 0.5).astype(int).flatten()
true_classes = test_generator.classes

# Generar el informe de clasificación
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("\nClassification Report:")
print(report)

In [ ]:
# Mostrar la matriz de confusión
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Visualización de la matriz de confusión
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, cmap='Blues')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_labels))
plt.xticks(tick_marks, class_labels, rotation=45)
plt.yticks(tick_marks, class_labels)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Añadir los números en la matriz de confusión
for i in range(len(class_labels)):
    for j in range(len(class_labels)):
        plt.text(j, i, str(conf_matrix[i, j]), ha='center', va='center', color='red')

plt.tight_layout()
plt.show()